## Community detection for all metropolitan areas

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [2]:
import pandas as pd
import numpy as np
import geopandas as gpd
import seaborn as sns
import networkx as nx

import scipy
import csv

import matplotlib.cm as cm
import matplotlib.pyplot as plt
import json
import community as community_louvain
from copy import deepcopy
# from modularity_maximization.utils import get_modularity

from itertools import product
import networkx.algorithms.community as nx_comm
from scipy.spatial.distance import pdist, squareform

%matplotlib inline

In [3]:
from oct2py import octave
#octave.addpath('/home/ubuntu/GenLouvain/')
#octave.addpath('/home/ubuntu/GenLouvain/private/')
_ = octave.addpath('/home/barcsab/urban_communities/scripts')
_ = octave.addpath('/home/ubuntu/GenLouvain/')
_ = octave.addpath('/home/ubuntu/GenLouvain/private/')

### data

In [4]:
# three networks - data IN
mobility = pd.read_csv("../data/usageousers_city_mobility_CT_networks.rpt.gz") ## A MOBILITY HÁLÓBÓL SZÁMOLOK POSITION-T, AZÉRT KELL!
follow_hh = pd.read_csv("../data/usageousers_city_follower_CT_HH_networks.rpt.gz")
follow_hh = follow_hh.rename(columns={"tract_home.1": "tract_home_1"})

# census tract name -> cbsacode
cbsacode = pd.read_csv("../data/cbsacode_shortname_tracts.csv",sep=";", index_col=0)
cbsacode['clean_name'] = cbsacode["short_name"].map(lambda s: s.split("/")[0].replace(' ','_').replace('.','').lower())

# census data
census = pd.read_csv("../data/censusdata_top50_2012.csv")
census_2 = pd.read_csv("../data/censusdata_top50_2017.csv")

# reading geojson data, converting it to geopandas dataframe
tract_geoms = gpd.GeoDataFrame.from_features(
    [json.loads(e.strip('\n')) for e in open('../data/censustract_geoms_top50.geojson').readlines()]
)

# Cartesian coordinate projection of tract centroids
tract_geoms['centroid'] = tract_geoms['geometry'].centroid
tract_center_dict = tract_geoms\
    .set_geometry('centroid',crs={'init':'epsg:4326'})\
    .to_crs({'init':'epsg:3785'})\
    .set_index('full_geoid')['centroid'].map(lambda p: p.coords[0]).to_dict()

/home/ubuntu/anaconda3/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [5]:
# unique lists for city names and city cbsacodes
city_l = cbsacode.clean_name.unique()
city_code_l = cbsacode.cbsacode.unique()

In [6]:
def create_graphs(city, g_type):
    """
    For a given city name, it generates a mobility and follower (home-home) graph.
    
    e.g. g_mob, g_fol_hh = create_graphs("Boston")
    
    It uses the previously loaded `mobility` and `follow_hh` pandas.DataFrames, in which
    the edges are listed for every city.
    
    Parameters:
    -----------
    city : str
        name of the city, see cbsacode dataframe -> clean_name
        
    g_type : str
        either "mobility" or "follow_hh"
        selects the type of graph to return
        
    Returns:
    --------
    
    g : networkx.Graph
        weighted undirected graph based on city name and g_type (e.g. follow_hh graph of Boston)
        
    """
    # city cbsacode based on name
    city_code = cbsacode[cbsacode.clean_name == city].iloc[0].cbsacode
    
    # select graph type
    if g_type == "mobility":
        # filtering large dataframes for the given city code
        mob_df = mobility[(mobility["cbsacode"] == city_code)&(mobility["tract_home"]!=mobility["tract_work"])]

        # create graphs
        # create empty graphs
        g_mob = nx.DiGraph() # mobility graph - weights are counts

        # fill in the networks with data
        mob_df['w_edges'] = list(zip(mob_df.tract_home,mob_df.tract_work,mob_df.cnt))
        g_mob.add_weighted_edges_from(mob_df["w_edges"], weight='cnt')

        # ineffective and slow!
        for e in g_mob.edges():
            r = (e[1],e[0])

            if r in g_mob.edges():
                c1 = g_mob.edges[e]['cnt']
                c2 = g_mob.edges[r]['cnt']

                g_mob.edges[e]['cnt'] = c1 + c2
                g_mob.edges[r]['cnt'] = c1 + c2

        # then let's convert the mobility graph to udirected
        g_mob = g_mob.to_undirected()

        g = g_mob
        
    elif g_type == "follow_hh":            
        # filtering large dataframes for the given city code
        fol_hh_df = follow_hh[(follow_hh["cbsacode"] == city_code)&(follow_hh["tract_home"]!=follow_hh["tract_home_1"])]

        # create graphs
        # create empty graphs
        g_fol_hh = nx.Graph() # follow home-home graph - weights are counts

        # this is an undirected graph already in the dataframe
        fol_hh_df['w_edges'] = list(zip(fol_hh_df.tract_home,fol_hh_df.tract_home_1,fol_hh_df.cnt))
        g_fol_hh.add_weighted_edges_from(fol_hh_df["w_edges"], weight='cnt')
        
        g = g_fol_hh
    
    return g

In [7]:
from time import time

In [8]:
# CONSENSUS CLUSTERING
def consen(city, algorithm_type, g_type):
    """
    Function that does the consensus clustering based on the results
    of multiple runs of previous algorithms.
    
    Parameters:
    -----------
    
    city : str
        cityname to runt he consensus clustering for (see cbsacode.clean_name)
    g_type : str
        either "mobility" or "follow_hh"
        selects the type of graph
        
    Returns:
    --------
    
    s_louv : dict
        tract_geoid -> partition label (int)
    """
    
    tic = time()
    
    print("Reading in necessary data...")
    csv = '../data/consensus_' + city + '_' + algorithm_type + '_' + g_type + '.csv'
    G = create_graphs(city, g_type)
    
    # results of multiple iterations from previous runs
    iters = pd.read_csv(csv)
    iters = iters.set_index('geoid')
    iters['clusts'] = iters.values.tolist()
    toc = time()
    print("Done.","%.2f" % (toc-tic))
    tic = toc

    print("Creating all possible node pairs...")
    # create all possible node pairs
    geoid_pairs = list(product(list(iters.index), list(iters.index)))
    consen_df = pd.DataFrame(geoid_pairs, columns=['geoid_1','geoid_2'])
    

    
    # remove selfloops
    consen_df = consen_df[consen_df.geoid_1!=consen_df.geoid_2]
    toc = time()
    print("Done.","%.2f" % (toc-tic))
    tic = toc
    
    print("Joining interation results to node pairs...")
    # joining iteration results as lists to both elements of the tract pair
    consen_df = pd.merge(consen_df, iters['clusts'], left_on = 'geoid_1', right_on = 'geoid')
    consen_df = pd.merge(consen_df, iters['clusts'], left_on = 'geoid_2', right_on = 'geoid')
    consen_df = consen_df.rename(columns = {'clusts_x': 'clusts_1', 'clusts_y': 'clusts_2'})
    toc = time()
    print("Done.","%.2f" % (toc-tic))
    tic = toc
    
    def same(row):
        """This function """
        return np.array(np.equal(row['clusts_1'], row['clusts_2'])).astype(int).sum()

    print("Counting same partitioning for node pairs...")
    # how many times are the two tracts (geoid_1 and geoid_2) clustered to the same community?
    # --> weights of a graph on which clustering gives the consensus clustering
    consen_df['w'] = consen_df.apply(lambda row: same(row), axis=1)
    toc = time()
    print("Done.","%.2f" % (toc-tic))
    tic = toc
    
    print("Last Louvain...")
    # graph for consensus clustering
    g_cons = nx.Graph() 
    consen_df['w_edges'] = list(zip(consen_df.geoid_1,consen_df.geoid_2,consen_df.w))
    g_cons.add_weighted_edges_from(consen_df["w_edges"], weight='w')

    # Louvain community detection 
    s_louv = community_louvain.best_partition(g_cons, weight='w')
    toc = time()
    print("Done.","%.2f" % (toc-tic))
    tic = toc
    
    return s_louv

In [9]:
consen('new_york','expert','follow_hh')

Reading in necessary data...


/home/ubuntu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Done. 0.95
Creating all possible node pairs...
Done. 7.92
Joining interation results to node pairs...
Done. 16.72
Counting same partitioning for node pairs...
Done. 983.96
Last Louvain...
Done. 296.76


{'14000US34017007100': 0,
 '14000US34017013800': 0,
 '14000US34003043001': 0,
 '14000US34029735103': 0,
 '14000US34003047400': 0,
 '14000US36081000100': 1,
 '14000US36061016900': 1,
 '14000US34017002300': 1,
 '14000US36005027900': 1,
 '14000US34013014000': 0,
 '14000US36061023200': 2,
 '14000US36061012700': 1,
 '14000US34039035700': 0,
 '14000US34025809302': 0,
 '14000US36047055300': 1,
 '14000US36047022900': 2,
 '14000US34031124500': 2,
 '14000US34031196402': 0,
 '14000US34017015002': 0,
 '14000US34017014400': 0,
 '14000US36061005700': 2,
 '14000US34013011700': 0,
 '14000US36047024100': 2,
 '14000US36047002100': 1,
 '14000US36047027500': 2,
 '14000US36061003700': 1,
 '14000US36119006700': 3,
 '14000US36119009400': 3,
 '14000US34017004102': 0,
 '14000US34017006500': 0,
 '14000US34025808800': 0,
 '14000US34029723200': 0,
 '14000US36047013100': 1,
 '14000US36047012901': 1,
 '14000US34013017700': 0,
 '14000US34039033500': 0,
 '14000US36061013500': 2,
 '14000US36081094202': 0,
 '14000US340

In [10]:
from copy import deepcopy

In [16]:
cons_clust_df =  deepcopy(cbsacode.set_index('geoid'))

In [ ]:
# g_mob = create_graphs('boston', 'mobility')
# ('boston', 'follow_hh')

In [12]:
# for example
city = "new_york"
g_type = "follow_hh"

# corresponding weighted undirected graph
G = create_graphs(city,g_type)

# TODO
# index conversion dicts
# for i,node_id in enumerate(G.nodes()):
# for elem in enumerate(["alma","korte"]):
#     print(elem)
# geoid -> integer 0-... N-1
# az elozo dict megforditottja
# int -> geoid

# storing iteration results, empty dataframe for nodes
consen_df = pd.DataFrame()
consen_df['geoid'] = G.nodes()

# consensus result
S_df = pd.DataFrame()

# TODO we should check if all nodes of the graph are in the tract_geom dataframe
# e.g. in create_graphs()
# if someone's not there, that is data error, print the tract_id, and leave the node out of the graph G
# only after this should we calculate the Expert input data

# dataprep for Expert algorithm
A = nx.adjacency_matrix(G)
coords = np.array([tract_center_dict[n] for n in G.nodes()])
d = pdist(coords)
D = squareform(pdist(coords))

# importance - number of user home in each tract
# TODO we should check if all nodes in the follow_hh graph have an importance!
# otherwise, the N... line is going to throw an error
tract_outdeg_mob = mobility.groupby('tract_home')[['cnt']].sum()
N = np.matrix([tract_outdeg_mob.loc[k].iloc[0] for k in G.nodes()]).T

# binsize
max_dist = np.amax(D)
b = max_dist/99 # number of bins = 100      
for _ in range(3):
    print(_)
    # TODO Eszter!!!! sometimes it gives an error in the first line
    # new thing: nout = 3
    Ms,Mgn,Dfn = octave.ModularitySpaGN(A,D,N,b, nout = 3)
    S,Q,n_it = octave.iterated_genlouvain(Ms, nout=3)
    S_df[len(S_df.columns)] = S.T[0]
    # TODO itt egy lepesben meg lehet cisnalni mindket algorithm_type-ot!!!

# TODO S_df["geoid"] = S_df.index.map(a_masodik_dicted)
# TODO mindket algorithm type-ra kimenteni a csv-t
S_df['geoid'] = list(G.nodes()) ## KERDES JO??? - szerintem igen (Eszter)
S_df = S_df.set_index('geoid')
csv_name = 'consensus_' + city + '_' + algorithm_type + '_' + g_type + '.csv'
S_df.to_csv('../data/'+ csv_name)

/home/ubuntu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


0



TypeError: cannot unpack non-iterable NoneType object

In [ ]:
# ==============================================================

# TODO atirni az uj fuggveny szerint
# de az uj fuggveny meg nem eleg gyors
S_cons = consen(csv,G) ## eddig ment HIBA, KÉRDÉS
consen_df['S_cons'] = S_cons
consen_df['city'] = city
consen_df['type'] = g_type
consen_df = consen_df.set_index('geoid')
pd.merge(cons_clust_df, consen_df, left_index = True, right_index = True)
cons_clust_df.to_csv('all_cons.csv')

In [ ]:
for city in city_l:
    g_mob, g_fol_hh = create_graphs(city, mobility, follow_hh)
    for (G, g_type) in [(g_mob, 'mob'), (g_fol_hh, 'fol_hh')]:
        consen_df = pd.DataFrame()
        consen_df['geoid'] = G.nodes()
        S_df = pd.DataFrame()
        A = nx.adjacency_matrix(G)
        coords = np.array([tract_center_dict[n] for n in G.nodes()]) ## KERDES ez jó így? tract_center_dict
        d = pdist(coords)
        D = squareform(pdist(coords))
        #### KERDES -  jo, ha az importance mindig a mobilityből van számolva??????
        # importance - number of users
        tract_outdeg_mob = mobility.groupby('tract_home')[['cnt']].sum()
        N = np.matrix([tract_outdeg_mob.loc[k].iloc[0] for k in G.nodes()]).T
        # binsize
        max_dist = np.amax(D)
        b = max_dist/99 # number of bins = 100      
        for _ in range(3):
            print(_)
            Ms,Mgn,Dfn = octave.ModularitySpaGN(A,D,N,b, nout = 3)
            S,Q,n_it = octave.iterated_genlouvain(Ms, nout=3)
            S_df[len(S_df.columns)] = S.T[0]
        S_df['geoid'] = list(G.nodes()) ## KERDES JO???
        S_df = S_df.set_index('geoid')
        csv_name = 'consensus' + city + '_' + g_type + '.csv'
        S_df.to_csv('../data/'+ csv_name)

/home/ubuntu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/home/ubuntu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


0


In [ ]:
        S_cons = consen(csv,G) ## eddig ment HIBA, KÉRDÉS
        consen_df['S_cons'] = S_cons
        consen_df['city'] = city
        consen_df['type'] = g_type
        consen_df = consen_df.set_index('geoid')
        pd.merge(cons_clust_df, consen_df, left_index = True, right_index = True)
cons_clust_df.to_csv('all_cons.csv')

In [ ]:
csv_city = str(G) + '_rawclust.csv'+ city
        with open(csv_city, mode='w') as clust_file:
            clust_writer = csv.writer(clust_file, delimiter=',', quotechar='"') ### ez kerdeses!!!!

   

    employee_writer.writerow(['John Smith', 'Accounting', 'November'])

In [106]:
tract_sum = cbsacode.groupby('cbsacode')[['cbsacode']].count() # number of tracts per city
tract_sum = tract_sum.rename(columns={'cbsacode': 'sum_tracts'})

In [102]:
# unique values in the network counted by side of edge

mob_u_h = mobility.tract_home.tolist()
mob_u_w = mobility.tract_work.tolist()
u_geoid_mob = set([*mob_u_h,*mob_u_w])
cbsacode['in_mob'] = cbsacode.geoid.isin(u_geoid_mob).astype(int)

fol_u_h = follow_hh.tract_home.tolist()
fol_u_h1 = follow_hh.tract_home_1.tolist()
u_geoid_fol = set([*fol_u_h,*fol_u_h1])
cbsacode['in_fol'] = cbsacode.geoid.isin(u_geoid_fol).astype(int)

In [113]:
netw_count = cbsacode.groupby('cbsacode')[['in_mob','in_fol']].sum()

In [114]:
city_df = pd.merge(tract_sum, netw_count, left_index=True, right_index=True)

In [ ]:
nx_comm.modularity()

G = nx.barbell_graph(3, 0)

nx_comm.modularity(G, [{0, 1, 2}, {3, 4, 5}])
0.35714285714285715

nx_comm.modularity(G, nx_comm.label_propagation_communities(G))
0.35714285714285715


In [115]:
city_df

,sum_tracts,in_mob,in_fol
cbsacode,,,
12060.0,947,947,940
12420.0,348,348,344
12580.0,674,674,666
13820.0,263,263,261
14460.0,990,990,983
15380.0,295,295,292
16740.0,536,536,528
16980.0,2201,2201,2176
17140.0,495,495,492


In [12]:
tract = mobility.groupby('tract_home')[['cnt']].sum()

In [13]:
tract

,cnt
tract_home,
14000US01007010001,9
14000US01007010002,14
14000US01007010003,10
14000US01007010004,22
14000US01009050101,21
14000US01009050102,29
14000US01009050200,20
14000US01009050300,12
14000US01009050400,1


In [ ]:
follow_hh = follow_hh.merge(tract_outdeg_mob, left_on='tract_home', right_index=True)

N = np.matrix([tract_outdeg_mob.loc[k].iloc[0] for k in g_fol_hh.nodes()]).T
# modify N

# deprecated